In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from geopy.geocoders import Nominatim
import time
import os

# Add predefined coordinates to use if geocoding fails
MARKET_COORDINATES = {
    'Atlanta': [33.7490, -84.3880],
    'Austin': [30.2672, -97.7431],
    'Baltimore': [39.2908, -76.6108],
    'Boston': [42.3601, -71.0589],
    'Charlotte': [35.2271, -80.8431],
    'Chicago': [41.8781, -87.6298],
    'Chicago Suburbs': [42.0470, -87.6846],
    'Dallas/Ft Worth': [32.7767, -96.7970],
    'Denver': [39.7392, -104.9903],
    'Detroit': [42.3315, -83.0466],
    'Houston': [29.7604, -95.3698],
    'Los Angeles': [34.0522, -118.2437],
    'Manhattan': [40.7831, -73.9712],
    'Nashville': [36.1627, -86.7816],
    'Northern New Jersey': [40.7357, -74.1724],
    'Northern Virginia': [38.8816, -77.0910],
    'Orange County': [33.7175, -117.8311],
    'Philadelphia': [39.9526, -75.1652],
    'Phoenix': [33.4484, -112.0740],
    'Raleigh/Durham': [35.7796, -78.6382],
    'Salt Lake City': [40.7608, -111.8910],
    'San Diego': [32.7157, -117.1611],
    'San Francisco': [37.7749, -122.4194],
    'Seattle': [47.6062, -122.3321],
    'South Bay/San Jose': [37.3382, -121.8863],
    'South Florida': [25.7617, -80.1918],
    'Southern Maryland': [38.5076, -76.9820],
    'Tampa': [27.9506, -82.4572],
    'Washington D.C.': [38.9072, -77.0369]
}

# Function to get latitude and longitude with retry mechanism
def get_coordinates(location_name):
    # First check if we have predefined coordinates
    if location_name in MARKET_COORDINATES:
        return MARKET_COORDINATES[location_name]
    
    # Try geocoding as fallback
    geolocator = Nominatim(user_agent="market_map")
    
    # Fix some location names for better geocoding
    location_map = {
        'Raleigh/Durham': 'Raleigh, NC, USA',
        'Dallas/Ft Worth': 'Dallas, TX, USA',
        'Manhattan': 'Manhattan, NY, USA',
        'Northern New Jersey': 'Newark, NJ, USA',
        'Northern Virginia': 'Arlington, VA, USA',
        'Orange County': 'Orange County, CA, USA',
        'South Bay/San Jose': 'San Jose, CA, USA',
        'South Florida': 'Miami, FL, USA',
        'Southern Maryland': 'Annapolis, MD, USA',
        'Chicago Suburbs': 'Evanston, IL, USA',
        'Washington D.C.': 'Washington DC, USA'
    }
    
    search_name = location_map.get(location_name, location_name + ', USA')
    
    for attempt in range(3):  # Try up to 3 times
        try:
            location = geolocator.geocode(search_name)
            if location:
                return location.latitude, location.longitude
            time.sleep(1)  # Be nice to the geocoding service
        except Exception as e:
            print(f"Error geocoding {location_name}: {e}")
            time.sleep(2)  # Wait a bit longer before retrying
    
    # Return default coordinates if geocoding fails
    print(f"Could not geocode {location_name}, using default coordinates")
    return None, None

# Function to calculate marker size with greater differentiation
def calculate_marker_size(rent_value, min_rent, max_rent):
    """
    Calculate a marker size that emphasizes differences in rent values.
    
    Parameters:
    -----------
    rent_value : float
        The rent value for this marker
    min_rent : float
        The minimum rent in the dataset
    max_rent : float
        The maximum rent in the dataset
        
    Returns:
    --------
    float : The calculated marker size
    """
    if pd.isna(rent_value) or min_rent == max_rent:
        return 15  # Default size
    
    # Normalize the value between 0 and 1
    normalized = (rent_value - min_rent) / (max_rent - min_rent)
    
    # Apply a power function to emphasize differences (square for higher contrast)
    # This makes high values much larger than low values
    emphasized = normalized ** 1.5
    
    # Map to a more dramatic size range (8 to 40)
    return 10 + emphasized * 40

# Function to create custom HTML with fullscreen capability and fixed slider position
def create_fullscreen_html(fig, filename="us_market_rent_fullscreen.html"):
    """Create an HTML file with fullscreen capabilities"""
    
    # First get the regular HTML from plotly
    raw_html = fig.to_html(
        config={
            'responsive': True,
            'displayModeBar': True,
            'scrollZoom': True
        },
        include_plotlyjs='cdn',
        full_html=True,
        include_mathjax=False
    )
    
    # Insert custom CSS for fullscreen and fixed control positioning
    custom_css = """
    <style>
    body, html {
        height: 100%;
        margin: 0;
        padding: 0;
        overflow: hidden;
    }
    
    .plotly-graph-div {
        height: 100vh !important;
        width: 100vw !important;
    }
    
    .js-plotly-plot, .plot-container {
        height: 100%;
        width: 100%;
    }
    
    /* Position controls for better layout */
    .updatemenu-container {
        top: 10px !important;
    }
    
    /* Much lower slider position to avoid covering any part of the map */
    .slider-container {
        bottom: 100px !important;
        padding-bottom: 50px !important;
    }

    /* Make slider more compact */
    .slider-track {
        height: 3px !important;
    }

    .slider-text {
        font-size: 10px !important;
    }
    </style>
    
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">
    """
    
    # Insert the custom CSS after the <head> tag
    modified_html = raw_html.replace('</head>', f'{custom_css}</head>')
    
    # Write the modified HTML to a file
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(modified_html)
    
    print(f"Fullscreen visualization saved as '{filename}'")
    
    # Try to open the file automatically in the default browser
    try:
        # Get absolute path for more reliable opening
        abs_path = os.path.abspath(filename)
        import webbrowser
        webbrowser.open('file://' + abs_path, new=2)
    except Exception as e:
        print(f"Could not auto-open the file: {e}")
    
    return filename

def create_time_series_visualization(csv_file_path=r'/Users/kaylienguyen/Downloads/Lease-revised.csv', use_sample_data=False):
    """
    Create an enhanced visualization with a time slider
    
    Parameters:
    -----------
    csv_file_path : str or None
        Path to the CSV file. If None, will use sample data
    use_sample_data : bool
        Whether to use sample data (True) or load actual data (False)
        
    Returns:
    --------
    None, displays the figure
    """
    if use_sample_data:
        # Create sample data that includes time dimension
        # Sample time periods - 2018-Q1 through 2024-Q4
        time_periods = []
        for year in range(2018, 2025):
            for quarter in range(1, 5):
                time_periods.append((year, quarter))
        
        # Sample markets
        markets = [
            'Atlanta', 'Austin', 'Boston', 'Chicago', 'Dallas/Ft Worth', 
            'Denver', 'Houston', 'Los Angeles', 'Manhattan', 'Miami', 
            'Orange County', 'Philadelphia', 'Phoenix', 'Raleigh/Durham', 
            'San Francisco', 'Seattle', 'Washington D.C.'
        ]
        
        # Generate data for each time period, market, and class
        data = []
        np.random.seed(42)  # For reproducibility
        
        # Define growth trends for different markets (annual percentage growth)
        market_growth = {
            'Atlanta': 5,
            'Austin': 8,
            'Boston': 4,
            'Chicago': 3,
            'Dallas/Ft Worth': 7,
            'Denver': 6,
            'Houston': 5,
            'Los Angeles': 4,
            'Manhattan': 3,
            'Miami': 9,
            'Orange County': 4,
            'Philadelphia': 3,
            'Phoenix': 7,
            'Raleigh/Durham': 6,
            'San Francisco': 2,
            'Seattle': 5,
            'Washington D.C.': 4
        }
        
        # Base rents for 2018-Q1 (starting values)
        base_rents = {
            'Atlanta': {'A': 29.5, 'O': 21.2},
            'Austin': {'A': 39.8, 'O': 26.5},
            'Boston': {'A': 50.2, 'O': 26.9},
            'Chicago': {'A': 39.5, 'O': 28.0},
            'Dallas/Ft Worth': {'A': 30.2, 'O': 23.1},
            'Denver': {'A': 34.7, 'O': 26.0},
            'Houston': {'A': 30.4, 'O': 18.8},
            'Los Angeles': {'A': 44.5, 'O': 36.6},
            'Manhattan': {'A': 77.2, 'O': 56.7},
            'Miami': {'A': 41.3, 'O': 28.5},
            'Orange County': {'A': 39.1, 'O': 33.6},
            'Philadelphia': {'A': 29.5, 'O': 24.5},
            'Phoenix': {'A': 26.8, 'O': 22.2},
            'Raleigh/Durham': {'A': 32.6, 'O': 26.5},
            'San Francisco': {'A': 70.5, 'O': 52.2},
            'Seattle': {'A': 44.8, 'O': 31.5},
            'Washington D.C.': {'A': 53.2, 'O': 40.7}
        }
        
        for time_idx, (year, quarter) in enumerate(time_periods):
            period_label = f"{year}-Q{quarter}"
            
            for market in markets:
                # Calculate quarters since start to simulate growth over time
                quarters_passed = time_idx
                
                # Apply baseline growth
                growth_factor = 1 + (market_growth[market] / 100 * quarters_passed / 4)
                
                # Add market-specific random variation
                market_random_factor = 1 + (np.random.random() * 0.04 - 0.02)  # +/- 2% random variation
                
                # Calculate rent for each class with growth over time
                for class_type in ['A', 'O']:
                    base_rent = base_rents[market][class_type]
                    current_rent = base_rent * growth_factor * market_random_factor
                    
                    # Get coordinates from the predefined dictionary
                    lat, lon = MARKET_COORDINATES.get(market, [None, None])
                    
                    data.append({
                        'year': year,
                        'quarter': quarter,
                        'time_period': period_label,
                        'market': market,
                        'internal_class': class_type,
                        'internal_class_rent': round(current_rent, 2),
                        'latitude': lat,
                        'longitude': lon
                    })
        
        df = pd.DataFrame(data)
        
    else:
        # Load actual data
        print(f"Loading data from {csv_file_path}...")
        try:
            leasing = pd.read_csv(csv_file_path)
            print(f"Successfully loaded {len(leasing)} rows of data.")
        except Exception as e:
            print(f"Error loading data: {e}")
            print("Using sample data instead.")
            return create_time_series_visualization(use_sample_data=True)
        
        # Filter out rows with missing internal_class_rent
        df = leasing.dropna(subset=['internal_class_rent'])
        
        # Keep only necessary columns and filter for classes A and O
        # Let's make sure we can handle different class naming in the data
        print(f"Unique internal classes: {leasing['internal_class'].unique()}")
        
        # Keep only specific classes - filter for A and O classes
        # If your data uses different class labels, modify this filter
        df = df[df['internal_class'].isin(['A', 'O'])]
        
        # Keep only necessary columns
        df = df[['year', 'quarter', 'market', 'internal_class', 'internal_class_rent']]
        
        # Create a time_period column (e.g., '2023-Q1')
        df['time_period'] = df['year'].astype(str) + '-Q' + df['quarter'].astype(str)
        
        # Get coordinates for each market (use predefined values if available)
        print("Getting coordinates for markets...")
        market_coords = {}
        markets = df['market'].unique()
        print(f"Markets in data: {sorted(markets)}")
        
        for market in markets:
            # Use predefined coordinates if available
            if market in MARKET_COORDINATES:
                market_coords[market] = MARKET_COORDINATES[market]
                continue
                
            # Only geocode markets we don't have predefined coordinates for
            if market not in market_coords:
                lat, lon = get_coordinates(market)
                market_coords[market] = (lat, lon)
                time.sleep(1)  # Respect rate limits
        
        # Add latitude and longitude
        df['latitude'] = df['market'].map(lambda x: market_coords.get(x, (None, None))[0])
        df['longitude'] = df['market'].map(lambda x: market_coords.get(x, (None, None))[1])
        
        # Drop rows with missing coordinates
        orig_len = len(df)
        df = df.dropna(subset=['latitude', 'longitude'])
        dropped = orig_len - len(df)
        if dropped > 0:
            print(f"Dropped {dropped} rows with missing coordinates.")
    
    # Get average rent by market, class, and time period
    df_avg = df.groupby(['time_period', 'year', 'quarter', 'market', 'internal_class']).agg({
        'internal_class_rent': 'mean',
        'latitude': 'first',
        'longitude': 'first'
    }).reset_index()
    
    # Sort by time period
    df_avg = df_avg.sort_values(['year', 'quarter', 'market', 'internal_class'])
    
    # Create a pivot table for each time period
    time_periods = df_avg['time_period'].unique()
    time_periods.sort()  # Sort time periods chronologically
    
    print(f"Time periods found: {time_periods}")
    
    # Find global min and max for consistent circle sizing
    a_class_rents = df_avg[df_avg['internal_class'] == 'A']['internal_class_rent']
    global_min_rent = a_class_rents.min() if len(a_class_rents) > 0 else 0
    global_max_rent = a_class_rents.max() if len(a_class_rents) > 0 else 100
    
    # Store frames for animation
    frames = []
    
    for period in time_periods:
        # Filter data for this time period
        period_data = df_avg[df_avg['time_period'] == period]
        
        # Skip if no data for this period
        if len(period_data) == 0:
            continue
            
        # Create pivot table for this period
        try:
            pivot_df = period_data.pivot_table(
                index='market',
                columns='internal_class',
                values='internal_class_rent',
                aggfunc='mean'
            ).reset_index()
            
            # Add coordinates
            pivot_df = pd.merge(
                pivot_df,
                period_data[['market', 'latitude', 'longitude']].drop_duplicates(),
                on='market'
            )
            
            # Ensure both A and O columns exist
            if 'A' not in pivot_df.columns:
                pivot_df['A'] = np.nan
            if 'O' not in pivot_df.columns:
                pivot_df['O'] = np.nan
                
            # Add marker size to dataframe
            pivot_df['marker_size'] = pivot_df['A'].apply(
                lambda x: calculate_marker_size(x, global_min_rent, global_max_rent)
            )
                
            # Enhanced hover text with marker size info
            pivot_df['hover_text'] = pivot_df.apply(
                lambda row: f"<b>{row['market']}</b><br>" +
                            f"Time Period: {period}<br>" +
                            f"Class A Rent: ${row['A']:.2f}<br>" +
                            f"Class O Rent: ${row['O']:.2f}" +
                            (f"<br>A/O Ratio: {(row['A']/row['O']):.2f}" if not pd.isna(row['A']) and not pd.isna(row['O']) and row['O'] > 0 else ""),
                axis=1
            )
            
            # Store this frame
            frames.append(go.Frame(
                data=[go.Scattergeo(
                    lat=pivot_df['latitude'],
                    lon=pivot_df['longitude'],
                    text=pivot_df['hover_text'],
                    hoverinfo='text',
                    mode='markers',
                    marker=dict(
                        size=pivot_df['marker_size'],
                        color=pivot_df['A'],
                        colorscale='Viridis',
                        cmin=global_min_rent,
                        cmax=global_max_rent,
                        colorbar=dict(
                            title='Class A Rent ($)',
                            thickness=15,
                            len=0.6,
                            y=0.5,
                            yanchor='middle'
                        ),
                        line=dict(width=1, color='DarkSlateGrey')
                    ),
                    name=period
                )],
                name=period,
                layout=go.Layout(
                    title=f"US Market Rental Rates Over Time: {period}"  # Title specific to this frame
                )
            ))
        except Exception as e:
            print(f"Error processing period {period}: {e}")
            continue
    
    # Check if we have any frames
    if len(frames) == 0:
        print("No frames could be created. Check your data and try again.")
        return None
        
    # Create base figure with first time period
    first_period = frames[0].name
    first_data = df_avg[df_avg['time_period'] == first_period]
    
    # Create pivot table for first period
    first_pivot = first_data.pivot_table(
        index='market',
        columns='internal_class',
        values='internal_class_rent',
        aggfunc='mean'
    ).reset_index()
    
    # Add coordinates
    first_pivot = pd.merge(
        first_pivot,
        first_data[['market', 'latitude', 'longitude']].drop_duplicates(),
        on='market'
    )
    
    # Ensure both A and O columns exist
    if 'A' not in first_pivot.columns:
        first_pivot['A'] = np.nan
    if 'O' not in first_pivot.columns:
        first_pivot['O'] = np.nan
    
    # Add marker size to dataframe
    first_pivot['marker_size'] = first_pivot['A'].apply(
        lambda x: calculate_marker_size(x, global_min_rent, global_max_rent)
    )
    
    # Create custom hover text
    first_pivot['hover_text'] = first_pivot.apply(
        lambda row: f"<b>{row['market']}</b><br>" +
                    f"Time Period: {first_period}<br>" +
                    f"Class A Rent: ${row['A']:.2f}<br>" +
                    f"Class O Rent: ${row['O']:.2f}" +
                    (f"<br>A/O Ratio: {(row['A']/row['O']):.2f}" if not pd.isna(row['A']) and not pd.isna(row['O']) and row['O'] > 0 else ""),
        axis=1
    )
    
    # Create figure with responsive layout
    fig = go.Figure(
        data=[go.Scattergeo(
            lat=first_pivot['latitude'],
            lon=first_pivot['longitude'],
            text=first_pivot['hover_text'],
            hoverinfo='text',
            mode='markers',
            marker=dict(
                size=first_pivot['marker_size'],
                color=first_pivot['A'],
                colorscale='Viridis',
                cmin=global_min_rent,
                cmax=global_max_rent,
                colorbar=dict(
                    title='Class A Rent ($)',
                    thickness=15,
                    len=0.6,
                    y=0.5,
                    yanchor='middle'
                ),
                line=dict(width=1, color='DarkSlateGrey')
            ),
            name=first_period
        )],
        frames=frames,
        layout=go.Layout(
            title=dict(
                text=f"US Market Rental Rates Over Time: {first_period}",
                x=0.5,
                y=0.98,
                yanchor='top',
                font=dict(size=20)
            ),
            updatemenus=[dict(
                type='buttons',
                showactive=False,
                y=0.90,  # Move buttons to top right
                x=0.9,
                xanchor='right',
                yanchor='top',
                pad=dict(r=10, t=10, b=10),
                buttons=[dict(
                    label='Play',
                    method='animate',
                    args=[None, dict(
                        frame=dict(duration=800, redraw=True),
                        fromcurrent=True,
                        transition=dict(duration=300, easing='quadratic-in-out')
                    )]
                ), dict(
                    label='Pause',
                    method='animate',
                    args=[[None], dict(
                        frame=dict(duration=0, redraw=True),
                        mode='immediate',
                        transition=dict(duration=0)
                    )]
                )]
            )],
            sliders=[dict(
                active=0,
                steps=[dict(
                    method='animate',
                    args=[[frame.name], dict(
                        mode='immediate',
                        frame=dict(duration=300, redraw=True),
                        transition=dict(duration=300),
                        layout=dict(title=f"US Market Rental Rates Over Time: {frame.name}")  # Update title when slider moves
                    )],
                    label=frame.name
                ) for frame in frames],
                transition=dict(duration=300, easing='cubic-in-out'),
                pad=dict(b=50, t=10),  # Much more bottom padding
                currentvalue=dict(
                    font=dict(size=14),
                    prefix='Time Period: ',
                    visible=True,
                    xanchor='right'
                ),
                len=0.9,
                x=0.1,
                y=-0.2,  # Position slider well below the map (-0.2 is way outside the plot area)
                yanchor='bottom',
                tickcolor='white'
            )],
            geo=dict(
                scope='usa',
                projection_type='albers usa',
                showland=True,
                landcolor='rgb(217, 217, 217)',
                coastlinecolor='white',
                countrycolor='white',
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                subunitcolor='white',
                center=dict(lat=39.5, lon=-98.35),  # Center the map slightly higher
                projection=dict(
                    scale=0.80  # Significantly reduced scale to leave more room
                )
            ),
            # Extreme bottom margin to accommodate the slider
            autosize=True,
            dragmode='pan',
            margin=dict(r=0, t=50, l=0, b=200, pad=0),  # Much larger bottom margin (200px)
        )
    )
    
    # Add annotations explaining the visualization
    fig.add_annotation(
        text="<b>Visualization Features:</b><br>" +
             "• Marker size shows Class A rent (larger = higher rent)<br>" +
             "• Color intensity also indicates rent level<br>" + 
             "• Hover over markers for detailed information",
        xref="paper", yref="paper",
        x=0.01, y=0.90,
        showarrow=False,
        bordercolor="black",
        borderwidth=1,
        borderpad=4,
        bgcolor="white",
        opacity=0.8
    )
    
    # Show figure
    fig.show()
    
    # Create custom fullscreen HTML
    create_fullscreen_html(fig)
    
    return fig

# Execute the function
if __name__ == "__main__":
    # Use actual data by default with the path provided
    try:
        fig = create_time_series_visualization(use_sample_data=False)
        if fig is None:
            print("Trying with sample data instead...")
            fig = create_time_series_visualization(use_sample_data=True)
    except Exception as e:
        print(f"Error: {e}")
        print("Trying with sample data instead...")
        fig = create_time_series_visualization(use_sample_data=True)

Loading data from /Users/kaylienguyen/Downloads/Lease-revised.csv...
Successfully loaded 194685 rows of data.
Unique internal classes: ['A' 'O' nan]
Getting coordinates for markets...
Markets in data: ['Atlanta', 'Austin', 'Baltimore', 'Boston', 'Charlotte', 'Chicago Suburbs', 'Detroit', 'Houston', 'Los Angeles', 'Manhattan', 'Nashville', 'Northern New Jersey', 'Northern Virginia', 'Philadelphia', 'Phoenix', 'Salt Lake City', 'San Diego', 'San Francisco', 'Seattle', 'South Florida', 'Tampa']
Time periods found: ['2018-Q1' '2018-Q2' '2018-Q3' '2018-Q4' '2019-Q1' '2019-Q2' '2019-Q3'
 '2019-Q4' '2020-Q1' '2020-Q2' '2020-Q3' '2020-Q4' '2021-Q1' '2021-Q2'
 '2021-Q3' '2021-Q4' '2022-Q1' '2022-Q2' '2022-Q3' '2022-Q4' '2023-Q1'
 '2023-Q2' '2023-Q3' '2023-Q4' '2024-Q1' '2024-Q2' '2024-Q3' '2024-Q4']


Fullscreen visualization saved as 'us_market_rent_fullscreen.html'
